# Analysis of SD Score & Prediction of Forward Looking Scores

This notebook will focus on analying the SD Score for stablecoins and not-stablecoin pools.
The notebook will import both data sets, analyze and visualize the data.
The analysis will include: 

1. Describing the data
2. Visualizing the data
3. Correlation analysis and matrix
4. Transformation of variables if needed

From the analysis we will determine if SD score is a potential predictor of forward looking scores and DeFi yields.

The predicition models used will include:

1. Linear Regression
2. Random Forest
3. Gradient Boosting

In [21]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns



In [22]:
# Importing Data
filepath_stables = '/Users/karolk/Python_Work/ML_Price/Datasets/DeFi_Quant_Data_stables.csv'
filepath_not_stables = '/Users/karolk/Python_Work/ML_Price/Datasets/DeFi_Quant_Data_n_stables.csv'
filepath_prices = '/Users/karolk/Python_Work/ML_Price/Datasets/asset_prices.csv'

stables = pd.read_csv(filepath_stables)
not_stables = pd.read_csv(filepath_not_stables)
prices = pd.read_csv(filepath_prices)

In [23]:
# getting head of stables
stables.head()

,chain,project,symbol,tvlUsd,apy,pool,stablecoin,ilRisk,exposure,outlier,...,tvlChange1d,SD_Score,SD_Score_7D_avg,SD_Score_7D_std,SD_Score_7D_forward,token_id_1,token_id_2,token_id_3,token_id_4,num_tokens
0,Ethereum,merkl,GHO-USDT,2172112.0,15.85675,00196622-42f0-461e-afa6-8887506876e6,True,no,multi,False,...,0.090731,65.020294,65.982986,0.941077,64.954019,GHO,USDT,NaN,NaN,2
1,Ethereum,merkl,GHO-USDT,2170674.0,15.83543,00196622-42f0-461e-afa6-8887506876e6,True,no,multi,False,...,-0.066203,64.773325,65.620267,0.825514,64.726852,GHO,USDT,NaN,NaN,2
2,Ethereum,merkl,GHO-USDT,2187678.0,15.70240,00196622-42f0-461e-afa6-8887506876e6,True,no,multi,False,...,0.783351,65.737994,65.523312,0.753064,64.314827,GHO,USDT,NaN,NaN,2
3,Ethereum,merkl,GHO-USDT,2190395.0,15.73245,00196622-42f0-461e-afa6-8887506876e6,True,no,multi,False,...,0.124196,66.022536,65.425676,0.603655,63.868409,GHO,USDT,NaN,NaN,2
4,Ethereum,merkl,GHO-USDT,2190395.0,15.73245,00196622-42f0-461e-afa6-8887506876e6,True,no,multi,False,...,0.000000,66.171875,65.426035,0.604172,63.473553,GHO,USDT,NaN,NaN,2


In [24]:
# getting head of not_stables
not_stables.head()

,chain,project,symbol,tvlUsd,apy,pool,stablecoin,ilRisk,exposure,outlier,...,tvlChange1d,SD_Score,SD_Score_7D_avg,SD_Score_7D_std,SD_Score_7D_forward,token_id_1,token_id_2,token_id_3,token_id_4,num_tokens
0,Base,uniswap-v3,ISK-WETH,113970.0,2.84529,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,...,-0.436796,12.634695,11.237825,1.015088,12.028860,ISK,WETH,NaN,NaN,2
1,Base,uniswap-v3,ISK-WETH,113706.0,0.58260,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,...,-0.231640,12.676294,11.545415,1.086171,12.115925,ISK,WETH,NaN,NaN,2
2,Base,uniswap-v3,ISK-WETH,112150.0,1.65160,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,...,-1.368441,12.774360,11.876214,1.052148,12.421818,ISK,WETH,NaN,NaN,2
3,Base,uniswap-v3,ISK-WETH,114564.0,1.54926,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,...,2.152474,12.522667,12.244316,0.630556,12.851304,ISK,WETH,NaN,NaN,2
4,Base,uniswap-v3,ISK-WETH,117558.0,1.51888,0005d7bf-1f14-4c74-92cd-857c9931053e,False,yes,multi,False,...,2.613386,11.492723,12.263053,0.601219,13.639087,ISK,WETH,NaN,NaN,2


In [25]:
# creating date column for the prices data
# Remove the 'Z' at the end of the string in the 'createdAt' column (if it exists)
prices['date_created'] = prices['date_created'].str.replace('Z', '')

# convert the 'date_created' column to datetime
prices['date_created'] = pd.to_datetime(prices['date_created'], format='ISO8601')

# create date column in the format of 'YYYY-MM-DD'
prices['date'] = prices['date_created'].dt.date

# drop the 'date_created' column
prices = prices.drop(columns=['date_created'])

In [26]:
# getting head of prices
prices.head()

# pivot the prices data for easier joining
prices = prices.pivot(index='date', columns='assetSlug', values='price')

# reset the index
prices = prices.reset_index()

# get all unique columns
unique_columns = prices.columns
unique_columns


Index(['date', 'agoric', 'akash', 'aleph-zero', 'aptos', 'archway',
       'asset-mantle', 'audius', 'aura-network', 'avalanche', 'axelar',
       'band-protocol', 'binance-smart-chain', 'cardano', 'casper-network',
       'celestia', 'celo', 'comdex', 'cosmos', 'crypto-com-coin', 'desmos',
       'dydx', 'e-money', 'elrond', 'ethereum-2-0', 'evmos', 'fantom',
       'fetch-ai', 'harmony', 'injective-protocol', 'irisnet', 'juno', 'kava',
       'kusama', 'matic-network', 'mina', 'near-protocol', 'ontology', 'orbs',
       'osmosis', 'persistence', 'polkadot', 'radix', 'regen',
       'secret-network', 'sei-network', 'sentinel', 'sifchain', 'skale',
       'solana', 'stafi-protocol', 'stargaze', 'sui', 'terra', 'terra-luna-2',
       'tezos', 'the-graph', 'tron', 'umee', 'zilliqa'],
      dtype='object', name='assetSlug')

In [27]:
# joining prices data to stables and not_stables matching on date
stables = stables.merge(prices, how='left', left_on='date', right_on='date')
not_stables = not_stables.merge(prices, how='left', left_on='date', right_on='date')

In [28]:
# creating a list of uniqe columns
columns = stables.columns

columns_to_keep = ['chain', 'project', 'symbol', 'tvlUsd', 'apy', 'pool', 'stablecoin',
                           'date', 'apyMean7d', 'apyStd7d',
                          'apyMean7dForward', 'tvlPercentile', 'apy7DPercentile',
                           'apy7DStdRatio', 'tvlChange7d', 'tvlChange1d',
                          'SD_Score', 'SD_Score_7D_avg', 'SD_Score_7D_std', 'SD_Score_7D_forward',
                          'token_id_1', 'token_id_2', 'token_id_3', 'token_id_4', 'num_tokens',
                         'akash', 'aleph-zero', 'aptos',
                          'archway', 'asset-mantle', 'audius', 'aura-network', 'avalanche',
                          'axelar', 'band-protocol', 'binance-smart-chain', 'cardano',
                          'casper-network', 'celestia', 'celo', 'comdex', 'cosmos',
                          'crypto-com-coin', 'desmos', 'dydx', 'e-money', 'elrond',
                          'ethereum-2-0', 'evmos', 'fantom', 'fetch-ai', 'harmony',
                          'injective-protocol', 'irisnet', 'juno', 'kava', 'kusama',
                          'matic-network', 'mina', 'near-protocol', 'ontology', 'orbs', 'osmosis',
                          'persistence', 'polkadot', 'radix', 'regen', 'secret-network',
                          'sei-network', 'sentinel', 'sifchain', 'skale', 'solana',
                          'stafi-protocol', 'stargaze', 'sui', 'terra', 'terra-luna-2', 'tezos',
                          'the-graph', 'tron', 'umee', 'zilliqa']

